In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
import pandas as pd 

In [13]:
df = pd.read_csv('/content/drive/My Drive/MLDevelopers/Submission_1/Question_Classification_Dataset.csv')
df

,Unnamed: 0,Questions,Category0,Category1,Category2
0,0,How did serfdom develop in and then leave Russ...,DESCRIPTION,DESC,manner
1,1,What films featured the character Popeye Doyle ?,ENTITY,ENTY,cremat
2,2,How can I find a list of celebrities ' real na...,DESCRIPTION,DESC,manner
3,3,What fowl grabs the spotlight after the Chines...,ENTITY,ENTY,animal
4,4,What is the full form of .com ?,ABBREVIATION,ABBR,exp
...,...,...,...,...,...
5447,5447,What 's the shape of a camel 's spine ?,ENTITY,ENTY,other
5448,5448,What type of currency is used in China ?,ENTITY,ENTY,currency
5449,5449,What is the temperature today ?,NUMERIC,NUM,temp
5450,5450,What is the temperature for cooking ?,NUMERIC,NUM,temp


In [14]:
df = df.drop(columns=['Unnamed: 0','Category1','Category2'])
df

,Questions,Category0
0,How did serfdom develop in and then leave Russ...,DESCRIPTION
1,What films featured the character Popeye Doyle ?,ENTITY
2,How can I find a list of celebrities ' real na...,DESCRIPTION
3,What fowl grabs the spotlight after the Chines...,ENTITY
4,What is the full form of .com ?,ABBREVIATION
...,...,...
5447,What 's the shape of a camel 's spine ?,ENTITY
5448,What type of currency is used in China ?,ENTITY
5449,What is the temperature today ?,NUMERIC
5450,What is the temperature for cooking ?,NUMERIC


In [15]:
#proses one hot encoding
category = pd.get_dummies(df.Category0)
df_baru = pd.concat([df,category],axis=1)
df_baru = df_baru.drop(columns='Category0')
df_baru

,Questions,ABBREVIATION,DESCRIPTION,ENTITY,HUMAN,LOCATION,NUMERIC
0,How did serfdom develop in and then leave Russ...,0,1,0,0,0,0
1,What films featured the character Popeye Doyle ?,0,0,1,0,0,0
2,How can I find a list of celebrities ' real na...,0,1,0,0,0,0
3,What fowl grabs the spotlight after the Chines...,0,0,1,0,0,0
4,What is the full form of .com ?,1,0,0,0,0,0
...,...,...,...,...,...,...,...
5447,What 's the shape of a camel 's spine ?,0,0,1,0,0,0
5448,What type of currency is used in China ?,0,0,1,0,0,0
5449,What is the temperature today ?,0,0,0,0,0,1
5450,What is the temperature for cooking ?,0,0,0,0,0,1


In [16]:
#mengubah nilai dari dataframe ke tipe data numpy array
quest = df_baru['Questions'].values
label = df_baru[['ABBREVIATION','DESCRIPTION','ENTITY','HUMAN','LOCATION','NUMERIC']].values

In [17]:
#membagi data untuk training dan testing
from sklearn.model_selection import train_test_split
quest_latih, quest_test, label_latih, label_test = train_test_split(quest, label, test_size=0.2)

In [18]:
#melakukan tokenisasi
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(quest_latih)
tokenizer.fit_on_texts(quest_test)

sekuens_latih = tokenizer.texts_to_sequences(quest_latih)
sekuens_test = tokenizer.texts_to_sequences(quest_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [31]:
#arsitektur model layer embedding dengan dimensi 16
import tensorflow as tf
model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(6, activation='softmax')])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [34]:
#membuat kelas callback
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>0.9):
      print("\nAkurasi telah mencapai >90%!")
      self.model.stop_training = True
callbacks = myCallback()

In [35]:
#melatih model
num_epochs = 30
history = model.fit(padded_latih, 
                    label_latih, 
                    epochs=num_epochs,
                    validation_data=(padded_test,label_test), 
                    verbose=2,
                    callbacks=[callbacks])

Epoch 1/30
137/137 - 3s - loss: 0.1500 - accuracy: 0.9496 - val_loss: 0.8163 - val_accuracy: 0.7984
Epoch 2/30
137/137 - 3s - loss: 0.0921 - accuracy: 0.9693 - val_loss: 0.8830 - val_accuracy: 0.7846
Epoch 3/30
137/137 - 3s - loss: 0.0631 - accuracy: 0.9844 - val_loss: 1.0584 - val_accuracy: 0.7910
Epoch 4/30
137/137 - 3s - loss: 0.0439 - accuracy: 0.9890 - val_loss: 1.0176 - val_accuracy: 0.8002
Epoch 5/30
137/137 - 3s - loss: 0.0390 - accuracy: 0.9890 - val_loss: 1.0904 - val_accuracy: 0.7910
Epoch 6/30
137/137 - 3s - loss: 0.0299 - accuracy: 0.9892 - val_loss: 1.2934 - val_accuracy: 0.7782
Epoch 7/30
137/137 - 3s - loss: 0.0209 - accuracy: 0.9938 - val_loss: 1.2475 - val_accuracy: 0.7938
Epoch 8/30
137/137 - 3s - loss: 0.0164 - accuracy: 0.9947 - val_loss: 1.4425 - val_accuracy: 0.7883
Epoch 9/30
137/137 - 3s - loss: 0.0220 - accuracy: 0.9947 - val_loss: 1.4410 - val_accuracy: 0.7764
Epoch 10/30
137/137 - 3s - loss: 0.0234 - accuracy: 0.9945 - val_loss: 1.3666 - val_accuracy: 0.7901